In [ ]:
import time

n = 7 # 3
startInfo = [1] * n
scopInfo  = [0] * n

valueOf = {"i": 1, "g": 0, "d": -1}
keys = ["idddgii", "idiidid", "iidddii", "iddgiig", "dgggiid", "didiigg", "gigddgg", "gdggggg", "gggiggd", "dgggddg"] # ["iid", "did", "gdg"]

# validate input
for key in keys:
    assert len(key) == n, f"Incorrect length of key: {key}"
    spareKey = key.replace("i", "").replace("g", "").replace("d", "")
    assert len(spareKey) == 0, f"The key '{key}' shoudn't contain '{spareKey}''"

def H (arr): return sum(arr)

def UseKey (arr, key):
    newKey = [0] * n
    for keyPoz in range (n): # len(key) = n for any key in keys
        newKey[keyPoz] = arr[keyPoz] + valueOf[key[keyPoz]]
        if (newKey[keyPoz] < 0): newKey[keyPoz] = 0 # tried to unlock the opened lock
    return newKey

In [ ]:
class Nod:
    nodes = []
    def __init__(self, info, key = None, g = 0, p = None):
        self.id = len(Nod.nodes)
        self.key = key
        self.info = info # unique
        self.p = p
        self.g = g
        self.h = H(info)
        self.f = self.g + self.h
        Nod.nodes.append(self)
        
    def __str__ (self):
        return (f"(Info={self.info}; UsedKey='{self.key}'; h={self.h}; g={self.g}; f={self.f})")
    def __repr__ (self):
        return (f"(Info={self.info}; UsedKey='{self.key}'; h={self.h}; g={self.g}; f={self.f})")

    def Succ (self):
        succ = []
        
        for key in keys:
            succ.append (Nod(UseKey(self.info, key), key, self.g + 1, self.id))

        return succ
    
    def HasPred (self, nod): # checks if self has nod as a predecesor
        predId = self.p
        while predId is not None:
            if Nod.nodes[predId].info == nod.info: return True
            predId = Nod.nodes[predId].p
        return False

    def IsInList (self, nods):
        for nodPoz in range(len(nods)):
            if nods[nodPoz].info == self.info: 
                return nodPoz
        return None

    def PrintPath (self):
        if self.p is not None: Nod.nodes[self.p].PrintPath()
        else: print ("Path:")
        print (self)

In [ ]:
open = [Nod (startInfo)]
closed = []
done = False
startTime = time.time()
while len(open) != 0 and not done:
    currNod = open.pop(0)
    closed.append (currNod)
    # if I was the final state I should have already stopped
    for s in currNod.Succ():
        if s.info == scopInfo:
            done = True
            s.PrintPath()
            break

        if currNod.HasPred(s): continue
        if s.IsInList(closed) or s.IsInList(open): continue

        poz = len(open)
        open.append (s)

        while poz > 0 and open[poz-1].f > open[poz].f:
            open[poz-1], open[poz] = open[poz], open[poz-1]
            poz -= 1

        while poz > 0 and open[poz-1].f == open[poz].f and open[poz-1].g > open[poz].g:
            open[poz-1], open[poz] = open[poz], open[poz-1]
            poz -= 1

print (f"Duration: {time.time() - startTime}")    
if not done:
    print("Lista open e vida, nu avem drum de la nodul start la nodul scop")

In [ ]:
import time

import pygame, sys


ADANCIME_MAX=6


def elem_identice(lista):
	mt=set(lista)
	if(len(mt)==1) :
		castigator = list(mt)[0]
		if castigator!=Joc.GOL:
			return castigator
		else :
			return False
	else :
		return False
		
def deseneaza_grid(display, tabla):
	w_gr=h_gr=50
	
	x_img = pygame.image.load('Pieces/blackQueen.png')
	x_img = pygame.transform.scale(x_img, (w_gr,h_gr))
	zero_img = pygame.image.load('Pieces/whiteQueen.png')
	zero_img = pygame.transform.scale(zero_img, (w_gr,h_gr))
	drt=[]
	for ind in range(len(tabla)):
		linie=ind//3
		coloana=ind%3
		patr = pygame.Rect(coloana*(w_gr), linie*(h_gr), w_gr, h_gr)
		print(str(coloana*(w_gr)), str(linie*(h_gr)))
		drt.append(patr)
		pygame.draw.rect(display, (255,255,255), patr)
		if tabla[ind]=='x':
			display.blit(x_img,(coloana*w_gr,linie*h_gr))
		elif tabla[ind]=='0':
			display.blit(zero_img,(coloana*w_gr,linie*h_gr))
	pygame.display.flip()
	return drt

class Joc:
	"""
	Clasa care defineste jocul. Se va schimba de la un joc la altul.
	"""
	NR_COLOANE=3
	JMIN=None
	JMAX=None
	GOL='#'
	def __init__(self, tabla=None):
		self.matr=tabla or [self.__class__.GOL]*9

		
	def final(self):
		rez= (elem_identice(self.matr[0:3]) 
		or elem_identice(self.matr[3:6]) 
		or elem_identice(self.matr[6:9])
		or elem_identice(self.matr[0:9:3])
		or elem_identice(self.matr[1:9:3])
		or elem_identice(self.matr[2:9:3])
		or elem_identice(self.matr[0:9:4])
		or elem_identice(self.matr[2:8:2]))
		if(rez):
			return rez
		elif self.__class__.GOL not in self.matr:
			return 'remiza'
		else:
			return False

	def mutari(self, jucator_opus):
		l_mutari=[]
		for i in range(len(self.matr)) :
			if self.matr[i]==self.__class__.GOL :
				matr_tabla_noua=list(self.matr)
				matr_tabla_noua[i]=jucator_opus
				l_mutari.append(Joc(matr_tabla_noua))
		return l_mutari
	

	#linie deschisa inseamna linie pe care jucatorul mai poate forma o configuratie castigatoare
	def linie_deschisa(self,lista, jucator):
		#obtin multimea simbolurilor de pe linie
		mt=set(lista)
		#verific daca sunt maxim 2 simboluri
		if(len(mt)<=2) :
			#daca multimea simbolurilor nu are alte simboluri decat pentru cel de "gol" si jucatorul curent
			if mt <= {self.__class__.GOL, jucator}:
				#inseamna ca linia este deschisa
				return 1
			else :
				return 0
		else :
			return 0
			
	def linii_deschise(self, jucator):
		return (self.linie_deschisa(self.matr[0:3],jucator) 
			+ self.linie_deschisa(self.matr[3:6], jucator) 
			+ self.linie_deschisa(self.matr[6:9], jucator)
			+ self.linie_deschisa(self.matr[0:9:3], jucator)
			+ self.linie_deschisa(self.matr[1:9:3], jucator)
			+ self.linie_deschisa(self.matr[2:9:3], jucator)
			+ self.linie_deschisa(self.matr[0:9:4], jucator) #prima diagonala
			+ self.linie_deschisa(self.matr[2:8:2], jucator)) # a doua diagonala
			
		
	def estimeaza_scor(self, adancime):
		t_final=self.final()
		#if (adancime==0):
		if t_final==self.__class__.JMAX :
			return (99+adancime)
		elif t_final==self.__class__.JMIN:
			return (-99-adancime)
		elif t_final=='remiza':
			return 0
		else:
			return (self.linii_deschise(self.__class__.JMAX)- self.linii_deschise(self.__class__.JMIN))
			


	def __str__(self):
		sir= (" ".join([str(x) for x in self.matr[0:3]])+"\n"+
		" ".join([str(x) for x in self.matr[3:6]])+"\n"+
		" ".join([str(x) for x in self.matr[6:9]])+"\n")
 
		return sir
			

class Stare:
	"""
	Clasa folosita de algoritmii minimax si alpha-beta
	Are ca proprietate tabla de joc
	Functioneaza cu conditia ca in cadrul clasei Joc sa fie definiti JMIN si JMAX (cei doi jucatori posibili)
	De asemenea cere ca in clasa Joc sa fie definita si o metoda numita mutari() care ofera lista cu configuratiile posibile in urma mutarii unui jucator
	"""
	def __init__(self, tabla_joc, j_curent, adancime, parinte=None, scor=None):
		self.tabla_joc=tabla_joc
		self.j_curent=j_curent
		
		#adancimea in arborele de stari
		self.adancime=adancime	
		
		#scorul starii (daca e finala) sau al celei mai bune stari-fiice (pentru jucatorul curent)
		self.scor=scor
		
		#lista de mutari posibile din starea curenta
		self.mutari_posibile=[]
		
		#cea mai buna mutare din lista de mutari posibile pentru jucatorul curent
		self.stare_aleasa=None

	def jucator_opus(self):
		if self.j_curent==Joc.JMIN:
			return Joc.JMAX
		else:
			return Joc.JMIN

	def mutari(self):		
		l_mutari=self.tabla_joc.mutari(self.j_curent)
		juc_opus=self.jucator_opus()
		l_stari_mutari=[Stare(mutare, juc_opus, self.adancime-1, parinte=self) for mutare in l_mutari]

		return l_stari_mutari
		
	
	def __str__(self):
		sir= str(self.tabla_joc) + "(Juc curent:"+self.j_curent+")\n"
		return sir
	

			
""" Algoritmul MinMax """

def min_max(stare):
	
	if stare.adancime==0 or stare.tabla_joc.final() :
		stare.scor=stare.tabla_joc.estimeaza_scor(stare.adancime)
		return stare
		
	#calculez toate mutarile posibile din starea curenta
	stare.mutari_posibile=stare.mutari()

	#aplic algoritmul minimax pe toate mutarile posibile (calculand astfel subarborii lor)
	mutari_scor=[min_max(mutare) for mutare in stare.mutari_posibile]
	


	if stare.j_curent==Joc.JMAX :
		#daca jucatorul e JMAX aleg starea-fiica cu scorul maxim
		stare.stare_aleasa=max(mutari_scor, key=lambda x: x.scor)
	else:
		#daca jucatorul e JMIN aleg starea-fiica cu scorul minim
		stare.stare_aleasa=min(mutari_scor, key=lambda x: x.scor)
	stare.scor=stare.stare_aleasa.scor
	return stare
	

def alpha_beta(alpha, beta, stare):
	if stare.adancime==0 or stare.tabla_joc.final() :
		stare.scor=stare.tabla_joc.estimeaza_scor(stare.adancime)
		return stare
	
	if alpha>beta:
		return stare #este intr-un interval invalid deci nu o mai procesez
	
	stare.mutari_posibile=stare.mutari()
		

	if stare.j_curent==Joc.JMAX :
		scor_curent=float('-inf')
		
		for mutare in stare.mutari_posibile:
			#calculeaza scorul
			stare_noua=alpha_beta(alpha, beta, mutare)
			
			if (scor_curent<stare_noua.scor):
				stare.stare_aleasa=stare_noua
				scor_curent=stare_noua.scor
			if(alpha<stare_noua.scor):
				alpha=stare_noua.scor
				if alpha>=beta:
					break

	elif stare.j_curent==Joc.JMIN :
		scor_curent=float('inf')
		
		for mutare in stare.mutari_posibile:
			
			stare_noua=alpha_beta(alpha, beta, mutare)
			
			if (scor_curent>stare_noua.scor):
				stare.stare_aleasa=stare_noua
				scor_curent=stare_noua.scor

			if(beta>stare_noua.scor):
				beta=stare_noua.scor
				if alpha>=beta:
					break
	stare.scor=stare.stare_aleasa.scor

	return stare
	






def afis_daca_final(stare_curenta):
	final=stare_curenta.tabla_joc.final()
	if(final):
		if (final=="remiza"):
			print("Remiza!")
		else:
			print("A castigat "+final)
			
		return True
		
	return False
		
	

def main():
	#initializare algoritm
	raspuns_valid=False
	while not raspuns_valid:
		tip_algoritm=input("Algorimul folosit? (raspundeti cu 1 sau 2)\n 1.Minimax\n 2.Alpha-beta\n ")
		if tip_algoritm in ['1','2']:
			raspuns_valid=True
		else:
			print("Nu ati ales o varianta corecta.")
	#initializare jucatori
	raspuns_valid=False
	while not raspuns_valid:
		Joc.JMIN=input("Doriti sa jucati cu x sau cu 0? ").lower()
		if (Joc.JMIN in ['x', '0']):
			raspuns_valid=True
		else:
			print("Raspunsul trebuie sa fie x sau 0.")
	Joc.JMAX= '0' if Joc.JMIN == 'x' else 'x'
	
	
	#initializare tabla
	tabla_curenta=Joc()
	print("Tabla initiala")
	print(str(tabla_curenta))
	
	#creare stare initiala
	stare_curenta=Stare(tabla_curenta,'x',ADANCIME_MAX)
	
	#setari interf grafica
	pygame.init()
	pygame.display.set_caption('x si 0')
	ecran=pygame.display.set_mode(size=(152,152))
	
	
	patratele=deseneaza_grid(ecran,tabla_curent

	matr) 	whie True :
		
		if (stare_curenta.j_curent==Joc.JMIN):
		#muta jucatorul
			for event in pygame.event.get():
				if event.type== pygame.QUIT:
					pygame.quit()
					sys.exit()
				if event.type == pygame.MOUSEBUTTONDOWN:
					
					pos = pygame.mouse.get_pos()
					
					for np in range(len(patratele)):
						if patratele[np].collidepoint(pos):
							linie=np//3
							coloana=np%3
							if stare_curenta.tabla_joc.matr[linie*3+coloana] == Joc.GOL:					
							
								#dupa iesirea din while sigur am valide atat linia cat si coloana
								#deci pot plasa simbolul pe "tabla de joc"
								stare_curenta.tabla_joc.matr[linie*3+coloana]=Joc.JMIN
								
								#afisarea starii jocului in urma mutarii utilizatorului
								print("\nTabla dupa mutarea jucatorului")
								print(str(stare_curenta))
								patratele=deseneaza_grid(ecran,stare_curenta.tabla_joc.matr)
								#testez daca jocul a ajuns intr-o stare finala
								#si afisez un mesaj corespunzator in caz ca da
								if (afis_daca_final(stare_curenta)):
									break
									
								#S-a realizat o mutare. Schimb jucatorul cu cel opus
								stare_curenta.j_curent=stare_curenta.jucator_opus()
								
		
		#--------------------------------
		else: #jucatorul e JMAX (calculatorul)
			#Mutare calculator
			
			#preiau timpul in milisecunde de dinainte de mutare
			t_inainte=int(round(time.time() * 1000))
			if tip_algoritm=='1':
				stare_actualizata=min_max(stare_curenta)
			else: #tip_algoritm==2
				stare_actualizata=alpha_beta(-500, 500, stare_curenta)
			stare_curenta.tabla_joc=stare_actualizata.stare_aleasa.tabla_joc
			print("Tabla dupa mutarea calculatorului")
			print(str(stare_curenta))
			
			patratele=deseneaza_grid(ecran,stare_curenta.tabla_joc.matr)
			#preiau timpul in milisecunde de dupa mutare
			t_dupa=int(round(time.time() * 1000))
			print("Calculatorul a \"gandit\" timp de "+str(t_dupa-t_inainte)+" milisecunde.")
			
			if (afis_daca_final(stare_curenta)):
				break
				
			#S-a realizat o mutare. Schimb jucatorul cu cel opus
			stare_curenta.j_curent=stare_curenta.jucator_opus()

			
if __name__ == "__main__" :
	main()
	while True :
		for event in pygame.event.get():
			if event.type== pygame.QUIT:
				pygame.quit()
				sys.exit()

In [ ]:
f = open("Pieces/blackQueen.png")

In [ ]:
import pygame
def deseneaza_grid(display, tabla):
	w_gr=h_gr=50
	drt=[]
	for ind in range(len(tabla)):
		linie=ind//3
		coloana=ind%3
		patr = pygame.Rect(8*(w_gr+1), 8*(h_gr+1), w_gr, h_gr)
		print(str(8*(w_gr+1)), str(8*(h_gr+1)))
		drt.append(patr)
		pygame.draw.rect(display, (255,255,255), patr)
	pygame.display.flip()
	return drt
pygame.init()
pygame.display.set_caption('x si 0')
ecran=pygame.display.set_mode(size=(152,152))
patratele=deseneaza_grid(ecran,[0]*8)
while True :
    for event in pygame.event.get():
        if event.type== pygame.QUIT:
            pygame.quit()
            sys.exit()

In [ ]:
pygame.init()

#set color with rgb
white,black,red = (255,255,255),(0,0,0),(255,0,0)

#set display
gameDisplay = pygame.display.set_mode((800,600))

#caption
pygame.display.set_caption("ChessBoard")

#beginning of logic
gameExit = False

lead_x = 20
lead_y = 20

while not gameExit:
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            gameExit = True

#For loop for chessboard 

#draw a rectangle
gameDisplay.fill(white)
pygame.draw.rect(gameDisplay, black, [lead_x,lead_y,20,20])
pygame.display.update()


#quit from pygame & python
pygame.quit()
quit()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
board = np.zeros((8,8))
board[1::2, 0::2] = 1
board[0::2, 1::2] = 1
plt.imshow(board, cmap='binary')
plt.show()

In [ ]:
import time

import pygame, sys


ADANCIME_MAX=6


def elem_identice(lista):
	mt=set(lista)
	if(len(mt)==1) :
		castigator = list(mt)[0]
		if castigator!=Joc.GOL:
			return castigator
		else :
			return False
	else :
		return False
		
def deseneaza_grid(display, tabla):
	w_gr=h_gr=50
	
	x_img = pygame.image.load('Pieces/blackQueen.png')
	x_img = pygame.transform.scale(x_img, (w_gr,h_gr))
	zero_img = pygame.image.load('Pieces/whiteQueen.png')
	zero_img = pygame.transform.scale(zero_img, (w_gr,h_gr))
	drt=[]
	for ind in range(len(tabla)):
		linie=ind//3
		coloana=ind%3
		patr = pygame.Rect(coloana*(w_gr), linie*(h_gr), w_gr, h_gr)
		print(str(coloana*(w_gr)), str(linie*(h_gr)))
		drt.append(patr)
		pygame.draw.rect(display, (255,255,255), patr)
		if tabla[ind]=='x':
			display.blit(x_img,(coloana*w_gr,linie*h_gr))
		elif tabla[ind]=='0':
			display.blit(zero_img,(coloana*w_gr,linie*h_gr))
	pygame.display.flip()
	return drt

class Joc:
	"""
	Clasa care defineste jocul. Se va schimba de la un joc la altul.
	"""
	NR_COLOANE=3
	JMIN=None
	JMAX=None
	GOL='#'
	def __init__(self, tabla=None):
		self.matr=tabla or [self.__class__.GOL]*9

		
	def final(self):
		rez= (elem_identice(self.matr[0:3]) 
		or elem_identice(self.matr[3:6]) 
		or elem_identice(self.matr[6:9])
		or elem_identice(self.matr[0:9:3])
		or elem_identice(self.matr[1:9:3])
		or elem_identice(self.matr[2:9:3])
		or elem_identice(self.matr[0:9:4])
		or elem_identice(self.matr[2:8:2]))
		if(rez):
			return rez
		elif self.__class__.GOL not in self.matr:
			return 'remiza'
		else:
			return False

	def mutari(self, jucator_opus):
		l_mutari=[]
		for i in range(len(self.matr)) :
			if self.matr[i]==self.__class__.GOL :
				matr_tabla_noua=list(self.matr)
				matr_tabla_noua[i]=jucator_opus
				l_mutari.append(Joc(matr_tabla_noua))
		return l_mutari
	

	#linie deschisa inseamna linie pe care jucatorul mai poate forma o configuratie castigatoare
	def linie_deschisa(self,lista, jucator):
		#obtin multimea simbolurilor de pe linie
		mt=set(lista)
		#verific daca sunt maxim 2 simboluri
		if(len(mt)<=2) :
			#daca multimea simbolurilor nu are alte simboluri decat pentru cel de "gol" si jucatorul curent
			if mt <= {self.__class__.GOL, jucator}:
				#inseamna ca linia este deschisa
				return 1
			else :
				return 0
		else :
			return 0
			
	def linii_deschise(self, jucator):
		return (self.linie_deschisa(self.matr[0:3],jucator) 
			+ self.linie_deschisa(self.matr[3:6], jucator) 
			+ self.linie_deschisa(self.matr[6:9], jucator)
			+ self.linie_deschisa(self.matr[0:9:3], jucator)
			+ self.linie_deschisa(self.matr[1:9:3], jucator)
			+ self.linie_deschisa(self.matr[2:9:3], jucator)
			+ self.linie_deschisa(self.matr[0:9:4], jucator) #prima diagonala
			+ self.linie_deschisa(self.matr[2:8:2], jucator)) # a doua diagonala
			
		
	def estimeaza_scor(self, adancime):
		t_final=self.final()
		#if (adancime==0):
		if t_final==self.__class__.JMAX :
			return (99+adancime)
		elif t_final==self.__class__.JMIN:
			return (-99-adancime)
		elif t_final=='remiza':
			return 0
		else:
			return (self.linii_deschise(self.__class__.JMAX)- self.linii_deschise(self.__class__.JMIN))
			


	def __str__(self):
		sir= (" ".join([str(x) for x in self.matr[0:3]])+"\n"+
		" ".join([str(x) for x in self.matr[3:6]])+"\n"+
		" ".join([str(x) for x in self.matr[6:9]])+"\n")
 
		return sir
			

class Stare:
	"""
	Clasa folosita de algoritmii minimax si alpha-beta
	Are ca proprietate tabla de joc
	Functioneaza cu conditia ca in cadrul clasei Joc sa fie definiti JMIN si JMAX (cei doi jucatori posibili)
	De asemenea cere ca in clasa Joc sa fie definita si o metoda numita mutari() care ofera lista cu configuratiile posibile in urma mutarii unui jucator
	"""
	def __init__(self, tabla_joc, j_curent, adancime, parinte=None, scor=None):
		self.tabla_joc=tabla_joc
		self.j_curent=j_curent
		
		#adancimea in arborele de stari
		self.adancime=adancime	
		
		#scorul starii (daca e finala) sau al celei mai bune stari-fiice (pentru jucatorul curent)
		self.scor=scor
		
		#lista de mutari posibile din starea curenta
		self.mutari_posibile=[]
		
		#cea mai buna mutare din lista de mutari posibile pentru jucatorul curent
		self.stare_aleasa=None

	def jucator_opus(self):
		if self.j_curent==Joc.JMIN:
			return Joc.JMAX
		else:
			return Joc.JMIN

	def mutari(self):		
		l_mutari=self.tabla_joc.mutari(self.j_curent)
		juc_opus=self.jucator_opus()
		l_stari_mutari=[Stare(mutare, juc_opus, self.adancime-1, parinte=self) for mutare in l_mutari]

		return l_stari_mutari
		
	
	def __str__(self):
		sir= str(self.tabla_joc) + "(Juc curent:"+self.j_curent+")\n"
		return sir
	

			
""" Algoritmul MinMax """

def min_max(stare):
	
	if stare.adancime==0 or stare.tabla_joc.final() :
		stare.scor=stare.tabla_joc.estimeaza_scor(stare.adancime)
		return stare
		
	#calculez toate mutarile posibile din starea curenta
	stare.mutari_posibile=stare.mutari()

	#aplic algoritmul minimax pe toate mutarile posibile (calculand astfel subarborii lor)
	mutari_scor=[min_max(mutare) for mutare in stare.mutari_posibile]
	


	if stare.j_curent==Joc.JMAX :
		#daca jucatorul e JMAX aleg starea-fiica cu scorul maxim
		stare.stare_aleasa=max(mutari_scor, key=lambda x: x.scor)
	else:
		#daca jucatorul e JMIN aleg starea-fiica cu scorul minim
		stare.stare_aleasa=min(mutari_scor, key=lambda x: x.scor)
	stare.scor=stare.stare_aleasa.scor
	return stare
	

def alpha_beta(alpha, beta, stare):
	if stare.adancime==0 or stare.tabla_joc.final() :
		stare.scor=stare.tabla_joc.estimeaza_scor(stare.adancime)
		return stare
	
	if alpha>beta:
		return stare #este intr-un interval invalid deci nu o mai procesez
	
	stare.mutari_posibile=stare.mutari()
		

	if stare.j_curent==Joc.JMAX :
		scor_curent=float('-inf')
		
		for mutare in stare.mutari_posibile:
			#calculeaza scorul
			stare_noua=alpha_beta(alpha, beta, mutare)
			
			if (scor_curent<stare_noua.scor):
				stare.stare_aleasa=stare_noua
				scor_curent=stare_noua.scor
			if(alpha<stare_noua.scor):
				alpha=stare_noua.scor
				if alpha>=beta:
					break

	elif stare.j_curent==Joc.JMIN :
		scor_curent=float('inf')
		
		for mutare in stare.mutari_posibile:
			
			stare_noua=alpha_beta(alpha, beta, mutare)
			
			if (scor_curent>stare_noua.scor):
				stare.stare_aleasa=stare_noua
				scor_curent=stare_noua.scor

			if(beta>stare_noua.scor):
				beta=stare_noua.scor
				if alpha>=beta:
					break
	stare.scor=stare.stare_aleasa.scor

	return stare

def afis_daca_final(stare_curenta):
	final=stare_curenta.tabla_joc.final()
	if(final):
		if (final=="remiza"):
			print("Remiza!")
		else:
			print("A castigat "+final)
			
		return True
		
	return False
		
	

def main():
	#initializare algoritm
	raspuns_valid=False
	while not raspuns_valid:
		tip_algoritm=input("Algorimul folosit? (raspundeti cu 1 sau 2)\n 1.Minimax\n 2.Alpha-beta\n ")
		if tip_algoritm in ['1','2']:
			raspuns_valid=True
		else:
			print("Nu ati ales o varianta corecta.")
	#initializare jucatori
	raspuns_valid=False
	while not raspuns_valid:
		Joc.JMIN=input("Doriti sa jucati cu x sau cu 0? ").lower()
		if (Joc.JMIN in ['x', '0']):
			raspuns_valid=True
		else:
			print("Raspunsul trebuie sa fie x sau 0.")
	Joc.JMAX= '0' if Joc.JMIN == 'x' else 'x'
	
	
	#initializare tabla
	tabla_curenta=Joc()
	print("Tabla initiala")
	print(str(tabla_curenta))
	
	#creare stare initiala
	stare_curenta=Stare(tabla_curenta,'x',ADANCIME_MAX)
	
	#setari interf grafica
	pygame.init()
	pygame.display.set_caption('x si 0')
	ecran=pygame.display.set_mode(size=(152,152))
	
	
	patratele=deseneaza_grid(ecran,tabla_curenta.matr)

	while not True:
		if (stare_curenta.j_curent==Joc.JMIN):
		#muta jucatorul
			for event in pygame.event.get():
				if event.type== pygame.QUIT:
					pygame.quit()
					sys.exit()
				if event.type == pygame.MOUSEBUTTONDOWN:
					
					pos = pygame.mouse.get_pos()
					
					for np in range(len(patratele)):
						if patratele[np].collidepoint(pos):
							linie=np//3
							coloana=np%3
							if stare_curenta.tabla_joc.matr[linie*3+coloana] == Joc.GOL:					
							
								#dupa iesirea din while sigur am valide atat linia cat si coloana
								#deci pot plasa simbolul pe "tabla de joc"
								stare_curenta.tabla_joc.matr[linie*3+coloana]=Joc.JMIN
								
								#afisarea starii jocului in urma mutarii utilizatorului
								print("\nTabla dupa mutarea jucatorului")
								print(str(stare_curenta))
								patratele=deseneaza_grid(ecran,stare_curenta.tabla_joc.matr)
								#testez daca jocul a ajuns intr-o stare finala
								#si afisez un mesaj corespunzator in caz ca da
								if (afis_daca_final(stare_curenta)):
									break
									
								#S-a realizat o mutare. Schimb jucatorul cu cel opus
								stare_curenta.j_curent=stare_curenta.jucator_opus()
								
		
		#--------------------------------
		else: #jucatorul e JMAX (calculatorul)
			#Mutare calculator
			
			#preiau timpul in milisecunde de dinainte de mutare
			t_inainte=int(round(time.time() * 1000))
			if tip_algoritm=='1':
				stare_actualizata=min_max(stare_curenta)
			else: #tip_algoritm==2
				stare_actualizata=alpha_beta(-500, 500, stare_curenta)
			stare_curenta.tabla_joc=stare_actualizata.stare_aleasa.tabla_joc
			print("Tabla dupa mutarea calculatorului")
			print(str(stare_curenta))
			
			patratele=deseneaza_grid(ecran,stare_curenta.tabla_joc.matr)
			#preiau timpul in milisecunde de dupa mutare
			t_dupa=int(round(time.time() * 1000))
			print("Calculatorul a \"gandit\" timp de "+str(t_dupa-t_inainte)+" milisecunde.")
			
			if (afis_daca_final(stare_curenta)):
				break
				
			#S-a realizat o mutare. Schimb jucatorul cu cel opus
			stare_curenta.j_curent=stare_curenta.jucator_opus()
			break

			
if __name__ == "__main__" :
	main()
	while True :
		for event in pygame.event.get():
			if event.type== pygame.QUIT:
				pygame.quit()
				sys.exit()

In [ ]:
import pygame
def deseneaza_grid(display, tabla):
	w_gr=h_gr=50
	
	x_img = pygame.image.load('Pieces/blackQueen.png')
	x_img = pygame.transform.scale(x_img, (w_gr,h_gr))
	zero_img = pygame.image.load('Pieces/whiteQueen.png')
	zero_img = pygame.transform.scale(zero_img, (w_gr,h_gr))
	drt=[]
	for ind in range(len(tabla)):
		linie=ind//3
		coloana=ind%3
		patr = pygame.Rect(coloana*(w_gr), linie*(h_gr), w_gr, h_gr)
		print(str(coloana*(w_gr)), str(linie*(h_gr)))
		drt.append(patr)
		pygame.draw.rect(display, (255,255,255), patr)
		if tabla[ind]=='x':
			display.blit(x_img,(coloana*w_gr,linie*h_gr))
		elif tabla[ind]=='0':
			display.blit(zero_img,(coloana*w_gr,linie*h_gr))
	pygame.display.flip()
	return drt

In [ ]:
pygame.init()
pygame.display.set_caption('x si 0')
ecran=pygame.display.set_mode(size=(152,152))
patratele=deseneaza_grid(ecran,[0]*9)

while True :
    for event in pygame.event.get():
        if event.type== pygame.QUIT:
            pygame.quit()
            sys.exit()

In [20]:
import time
from keyboard import is_pressed

valueOf = {"i": 1, "g": 0, "d": -1} # where "i" = lock one more time, "g" = does not modify it, "d" = unlock it once if it in not unlocked

def readKeys (inputFileName):
  
    outputFileName = inputFileName.replace("input", "outputN")
    assert outputFileName != inputFileName, "Incorrect input file name, must include 'input'"
    try:
        inputFile  = open (inputFileName, "r")
    except IOError:
        raise Exception("Problem with input file " + inputFileName)
        
    global outputFile, keys, n, startInfo, scopInfo
    outputFile = open (outputFileName, "w")

    startInfo = []
    for x in inputFile.readline().replace('\n', '').split(' '):
        startInfo.append(int(x))
    keys = inputFile.read().replace('\n', '').split(" ") 
    inputFile.close()

    n = len(startInfo)
    assert n > 0, "The number of locks cannot be 0"
    assert len(keys) > 0, "No keys were given"
    openable  = [False] * n
    scopInfo  = [0] * n # I want to unlock all the locks

    # validate input
    for key in keys:
        assert len(key) == n, f"Incorrect length of key: {key}"
        spareKey = key.replace("i", "").replace("g", "").replace("d", "")
        assert len(spareKey) == 0, f"The key '{key}' shoudn't contain '{spareKey}''"
        for i in range(n):
            if key[i] == 'd': 
                openable[i] = True
    assert sum(openable) == n, f"The locks that cannot be oppend are marked with false: {openable}"

"""
    In some cases the keys are strongly connected and there is no solution
    Example: ["idgdgii", "idgidid", "gidddii", "idggiig", "dgigidd"]
    In that example, the program might never stop (or ran out of memory)
    A solution that might work (not implemented in this code, not sure it will work):
        Create a graph for each lockA where the nodes are the keys and there is an edge from key1 to key2 if key1 locks lockA and key2 unlocks it
        Then we should find intergraphs dependencies and before we decide to add a node in the open, to test for starvation or deadlock
    Simple examples: 
        1. keys=[0:'id', 1:'di']
            Graph lock0 dependecies: key0->key1 
            Graph lock1 dependecies: key1->key0
        2. keys=[0:'idg', 1:'gid', 2:'dgi']
            Graph lock0 dependecies: key0->key2 
            Graph lock1 dependecies: key1->key0
            Graph lock1 dependecies: key2->key1
        Those examples are small and all posibilites will be exhausted fast, but on more complex ones it will take a while
    A multiple graph is needed because it is important on which lock 2 keys are dependent
"""

def Hmax (arr): return max(arr) # the times we need to unlocked de most locked lock (doesn't care about the other locks) - does not overestimate 
def Hsum (arr): return sum(arr) # takes into account all locks - overestimates, but it a lot better than the Hmax
def Hlocked (arr): return sum(1 for k in arr if k != 0) # the number of not unlocked locks - overestimates, but not as much as Hsum
def HH (arr): 
    s = m = 0
    for x in arr:
        s += x
        if x > m: m = x
    return [m, s]

'''
    The problem with Hmax is that if the lock state is the following [0, 1, 2, 0, 100, 2, 0, 4] or [0, 10, 20, 0, 100, 20, 0, 40], it will evaluate them the same (100)
        - even if it does not overestimates
        - the other locks are important too and should be taken into account (not only the maximum)
    The problem with Hlocked is that if the lock state is the following [0, 1, 2, 0, 100, 2, 0, 4] or [0, 10, 20, 0, 100, 20, 0, 40] or [0, 1, 1, 0, 1, 1, 0, 1], it will evaluate them the same (5)
        - it overestimates the solution because after using the key 'gddgddgd' on [0, 1, 1, 0, 1, 1, 0, 1] it is in the final state after one key used, not 5
        - the number of how many times a lock is locked is important too and should be taken into account (not only if it is unlocked or not)
        - the range of values is limited to [0, n]
    The problem with Hsum is that it overestimates the solution because after using the key 'gddgddgd' on [0, 1, 1, 0, 1, 1, 0, 1] it is in the final state after one key used, not 5
    BUT it knows the huge difference between [0, 1, 2, 0, 100, 2, 0, 4] or [0, 10, 20, 0, 100, 20, 0, 40] or [0, 1, 1, 0, 1, 1, 0, 1]

    Best solution that does not overestimates: the combination of Hmax and Hsum
        we will sort them after Hmax which does not overestimates and if two Hmax are the same, after Hsum

'''

def UseKey (arr, key): # returns None if the current lock was better without using this key (if all unlocked locks were already unlocked)
    newKey = [0] * n
    alLeastOneUnlocked = False
    for keyPoz in range (n): # len(key) = n for any key in keys
        newKey[keyPoz] = arr[keyPoz] + valueOf[key[keyPoz]]
        if (newKey[keyPoz] < 0): newKey[keyPoz] = 0 # tried to unlock the unlocked lock
        else: alLeastOneUnlocked = True # at least one lock was truthly unlocked once
    return newKey if alLeastOneUnlocked else None

class Nod:
    nodes = []
    heuristic = Hsum
    def __init__(self, info, key = None, g = 0, p = None):
        self.id = len(Nod.nodes)
        self.key = key
        self.info = info # unique
        self.p = p
        self.g = g
        self.h = Nod.heuristic(info)
        if isinstance(self.h, list): # only when you use HH
            self.f = [x+self.g for x in self.h]
        else: # Hmax, Hsum, Hlocked
            self.f = self.g + self.h # for Hmax, Hsum, Hlocked

        Nod.nodes.append(self)
        
    def __str__ (self):
        return (f"(Info={self.info}; UsedKey='{self.key}'; h={self.h}; g={self.g}; f={self.f})")
    def __repr__ (self):
        return (f"(Info={self.info}; UsedKey='{self.key}'; h={self.h}; g={self.g}; f={self.f})")

    def Succ (self):
        succ = []
        
        for key in keys:
            info = UseKey(self.info, key)
            if info: # all the keys that unlocks at least one lock
                succ.append (Nod(info, key, self.g + 1, self.id))

        return succ
    
    def HasPred (self, nod): # checks if self has nod as a predecesor
        predId = self.p
        while predId is not None:
            if Nod.nodes[predId].info == nod.info: return True
            predId = Nod.nodes[predId].p
        return False

    def IsInList (self, nods):
        for nodPoz in range(len(nods)):
            
            # if nods[nodPoz].info == self.info: 
            #     return nodPoz

            atLeastOneBetter = False
            for i in range(n):
                if self.info[i] < nods[nodPoz].info[i]:
                    atLeastOneBetter = True        
                    break
            if not atLeastOneBetter: 
                return nodPoz #if nods[nodPoz].info == self.info else None
        return None

    def PrintPath (self):
        if self.p is not None: Nod.nodes[self.p].PrintPath()
        else: print ("Path:", file=outputFile)
        print (self, file=outputFile)

# this function is the impementaiton from the course of A*, any comments will be redundant
def solve(h = Hsum): # h is the used heuristic function
    Nod.heuristic = h   
    opened = [Nod (startInfo)]
    closed = []
    done = False
    if startInfo == scopInfo:
        opened[0].PrintPath()
        done = True
    print ("\tpress q to stop the current execution")
    startTime = time.time()
    while len(opened) != 0 and not done:
        currNod = opened.pop(0)
        closed.append (currNod)
        # if I was the final state I should have already stopped
        for s in currNod.Succ():
            if s.info == scopInfo:
                done = True
                s.PrintPath()
                break

            if currNod.HasPred(s): continue
            if s.IsInList(closed) or s.IsInList(opened): continue
            alLeastOneBetter = False
            for i in range(n):
                if currNod.info[i] > s.info[i]:
                    alLeastOneBetter = True
            
            if not alLeastOneBetter: continue

            poz = len(opened)
            opened.append (s)

            while poz > 0 and opened[poz-1].f > opened[poz].f:
                opened[poz-1], opened[poz] = opened[poz], opened[poz-1]
                poz -= 1

            while poz > 0 and opened[poz-1].f == opened[poz].f and opened[poz-1].g > opened[poz].g:
                opened[poz-1], opened[poz] = opened[poz], opened[poz-1]
                poz -= 1

            if is_pressed('q'): # if it takes too long and you want to stop it
                invalidRsp = True
                while invalidRsp:
                    try:
                        rsp = int(input("Press 0 to exit or 1 to continue"))
                        if rsp == 1:
                            break
                        if rsp == 0:
                            done = True
                            print ("\tForce exit")
                            print ("Force exit", file=outputFile)
                            break
                        print(f'Invalid choice (choose between 0 and 1).')                            
                    except ValueError:
                            print(f'Invalid choice (choose between 0 and 1).')

    print (f"Duration: {time.time() - startTime}", file=outputFile)    
    if not done:
        print("Unsolvable", file=outputFile)

def main():
    inputFilesName = ["232_Surcea_Mihai-Daniel_Lab6_Pb8_input1.txt", "232_Surcea_Mihai-Daniel_Lab6_Pb8_input2.txt", "232_Surcea_Mihai-Daniel_Lab6_Pb8_input3.txt", "232_Surcea_Mihai-Daniel_Lab6_Pb8_input4.txt", "232_Surcea_Mihai-Daniel_Lab6_Pb8_input5.txt", "232_Surcea_Mihai-Daniel_Lab6_Pb8_input6.txt"]
    for inputFileName in inputFilesName:
        print ("\nUsing", inputFileName)
        readKeys(inputFileName)
        print ("\n\tUsing Hlocked")
        print ("\nUsing Hlocked", file=outputFile)
        solve (Hlocked)
        print ("\n\tUsing Hsum")
        print ("\nUsing Hsum", file=outputFile)
        solve (Hsum)
        print ("\n\tUsing Hmax")
        print ("\nUsing Hmax", file=outputFile)
        solve (Hmax)
        print ("\n\tUsing HH")
        print ("\nUsing HH", file=outputFile)
        solve (HH)
        outputFile.close()
        print("\nDone")

if __name__ == "__main__":
    main()
_Pb8_input5.txt", "232_Surcea_Mihai-Daniel_Lab6_Pb8_input6.txt"]
    for inputFileName in inputFilesName:
        print ("\nUsing", inputFileName)
        readKeys(inputFileName)
        print ("\n\tUsing Hlocked")
        print ("\nUsing Hlocked", file=outputFile)
        solve (Hlocked)
        print ("\n\tUsing Hsum")
        print ("\nUsing Hsum", file=outputFile)
        solve (Hsum)
        print ("\n\tUsing Hmax")
        print ("\nUsing Hmax", file=outputFile)
        solve (Hmax)
        print ("\n\tUsing HH")
        print ("\nUsing HH", file=outputFile)
        solve (HH)
        outputFile.close()

if __name__ == "__main__":
    main()



Using 232_Surcea_Mihai-Daniel_Lab6_Pb8_input2.txt

	Using Hlocked
	press q to stop the current execution

	Using Hsum
	press q to stop the current execution

	Using Hmax
	press q to stop the current execution

	Using HH
	press q to stop the current execution

Using 232_Surcea_Mihai-Daniel_Lab6_Pb8_input3.txt

	Using Hlocked
	press q to stop the current execution

	Using Hsum
	press q to stop the current execution

	Using Hmax
	press q to stop the current execution

	Using HH
	press q to stop the current execution

Using 232_Surcea_Mihai-Daniel_Lab6_Pb8_input4.txt

	Using Hlocked
	press q to stop the current execution

	Using Hsum
	press q to stop the current execution

	Using Hmax
	press q to stop the current execution

	Using HH
	press q to stop the current execution

Using 232_Surcea_Mihai-Daniel_Lab6_Pb8_input5.txt

	Using Hlocked
	press q to stop the current execution

	Using Hsum
	press q to stop the current execution

	Using Hmax
	press q to stop the current execution

	Using HH

In [20]:
import time
from keyboard import is_pressed

valueOf = {"i": 1, "g": 0, "d": -1} # where "i" = lock one more time, "g" = does not modify it, "d" = unlock it once if it in not unlocked

def readKeys (inputFileName):
  
    outputFileName = inputFileName.replace("input", "output")
    assert outputFileName != inputFileName, "Incorrect input file name, must include 'input'"
    try:
        inputFile  = open (inputFileName, "r")
    except IOError:
        raise Exception("Problem with input file " + inputFileName)
        
    global outputFile, keys, n, startInfo, scopInfo
    outputFile = open (outputFileName, "w")

    startInfo = []
    for x in inputFile.readline().replace('\n', '').split(' '):
        startInfo.append(int(x))
    keys = inputFile.read().replace('\n', '').split(" ") 
    inputFile.close()

    n = len(startInfo)
    assert n > 0, "The number of locks cannot be 0"
    assert len(keys) > 0, "No keys were given"
    openable  = [False] * n
    scopInfo  = [0] * n # I want to unlock all the locks

    # validate input
    for key in keys:
        assert len(key) == n, f"Incorrect length of key: {key}"
        spareKey = key.replace("i", "").replace("g", "").replace("d", "")
        assert len(spareKey) == 0, f"The key '{key}' shoudn't contain '{spareKey}''"
        for i in range(n):
            if key[i] == 'd': 
                openable[i] = True
    assert sum(openable) == n, f"The locks that cannot be oppend are marked with false: {openable}"

"""
    In some cases the keys are strongly connected and there is no solution
    Example: ["idgdgii", "idgidid", "gidddii", "idggiig", "dgigidd"]
    In that example, the program might never stop (or ran out of memory)
    A solution that might work (not implemented in this code, not sure it will work):
        Create a graph for each lockA where the nodes are the keys and there is an edge from key1 to key2 if key1 locks lockA and key2 unlocks it
        Then we should find intergraphs dependencies and before we decide to add a node in the open, to test for starvation or deadlock
    Simple examples: 
        1. keys=[0:'id', 1:'di']
            Graph lock0 dependecies: key0->key1 
            Graph lock1 dependecies: key1->key0
        2. keys=[0:'idg', 1:'gid', 2:'dgi']
            Graph lock0 dependecies: key0->key2 
            Graph lock1 dependecies: key1->key0
            Graph lock1 dependecies: key2->key1
        Those examples are small and all posibilites will be exhausted fast, but on more complex ones it will take a while
    A multiple graph is needed because it is important on which lock 2 keys are dependent
"""

def Hmax (arr): return max(arr) # the times we need to unlocked de most locked lock (doesn't care about the other locks) - does not overestimate 
def Hsum (arr): return sum(arr) # takes into account all locks - overestimates, but it a lot better than the Hmax
def Hlocked (arr): return sum(1 for k in arr if k != 0) # the number of not unlocked locks - overestimates, but not as much as Hsum
def HH (arr): 
    s = m = 0
    for x in arr:
        s += x
        if x > m: m = x
    return [m, s]

'''
    The problem with Hmax is that if the lock state is the following [0, 1, 2, 0, 100, 2, 0, 4] or [0, 10, 20, 0, 100, 20, 0, 40], it will evaluate them the same (100)
        - even if it does not overestimates
        - the other locks are important too and should be taken into account (not only the maximum)
    The problem with Hlocked is that if the lock state is the following [0, 1, 2, 0, 100, 2, 0, 4] or [0, 10, 20, 0, 100, 20, 0, 40] or [0, 1, 1, 0, 1, 1, 0, 1], it will evaluate them the same (5)
        - it overestimates the solution because after using the key 'gddgddgd' on [0, 1, 1, 0, 1, 1, 0, 1] it is in the final state after one key used, not 5
        - the number of how many times a lock is locked is important too and should be taken into account (not only if it is unlocked or not)
        - the range of values is limited to [0, n]
    The problem with Hsum is that it overestimates the solution because after using the key 'gddgddgd' on [0, 1, 1, 0, 1, 1, 0, 1] it is in the final state after one key used, not 5
    BUT it knows the huge difference between [0, 1, 2, 0, 100, 2, 0, 4] or [0, 10, 20, 0, 100, 20, 0, 40] or [0, 1, 1, 0, 1, 1, 0, 1]

    Best solution that does not overestimates: the combination of Hmax and Hsum
        we will sort them after Hmax which does not overestimates and if two Hmax are the same, after Hsum

'''

def UseKey (arr, key): # returns None if the current lock was better without using this key (if all unlocked locks were already unlocked)
    newKey = [0] * n
    alLeastOneUnlocked = False
    for keyPoz in range (n): # len(key) = n for any key in keys
        newKey[keyPoz] = arr[keyPoz] + valueOf[key[keyPoz]]
        if (newKey[keyPoz] < 0): newKey[keyPoz] = 0 # tried to unlock the unlocked lock
        else: alLeastOneUnlocked = True # at least one lock was truthly unlocked once
    return newKey if alLeastOneUnlocked else None

class Nod:
    nodes = []
    heuristic = Hsum
    def __init__(self, info, key = None, g = 0, p = None):
        self.id = len(Nod.nodes)
        self.key = key
        self.info = info # unique
        self.p = p
        self.g = g
        self.h = Nod.heuristic(info)
        if isinstance(self.h, list): # only when you use HH
            self.f = [x+self.g for x in self.h]
        else: # Hmax, Hsum, Hlocked
            self.f = self.g + self.h # for Hmax, Hsum, Hlocked

        Nod.nodes.append(self)
        
    def __str__ (self):
        return (f"(Info={self.info}; UsedKey='{self.key}'; h={self.h}; g={self.g}; f={self.f})")
    def __repr__ (self):
        return (f"(Info={self.info}; UsedKey='{self.key}'; h={self.h}; g={self.g}; f={self.f})")

    def Succ (self):
        succ = []
        
        for key in keys:
            info = UseKey(self.info, key)
            if info: # all the keys that unlocks at least one lock
                succ.append (Nod(info, key, self.g + 1, self.id))

        return succ
    
    def HasPred (self, nod): # checks if self has nod as a predecesor
        predId = self.p
        while predId is not None:
            if Nod.nodes[predId].info == nod.info: return True
            predId = Nod.nodes[predId].p
        return False

    def IsInList (self, nods):
        for nodPoz in range(len(nods)):
            
            # if nods[nodPoz].info == self.info: 
            #     return nodPoz

            atLeastOneBetter = False
            for i in range(n):
                if self.info[i] < nods[nodPoz].info[i]:
                    atLeastOneBetter = True        
                    break
            if not atLeastOneBetter: 
                return nodPoz #if nods[nodPoz].info == self.info else None
        return None

    def PrintPath (self):
        if self.p is not None: Nod.nodes[self.p].PrintPath()
        else: print ("Path:", file=outputFile)
        print (self, file=outputFile)

# this function is the impementaiton from the course of A*, any comments will be redundant
def solve(h = Hsum): # h is the used heuristic function
    Nod.heuristic = h   
    opened = [Nod (startInfo)]
    closed = []
    done = False
    if startInfo == scopInfo:
        opened[0].PrintPath()
        done = True
    print ("\tpress q to stop the current execution")
    startTime = time.time()
    while len(opened) != 0 and not done:
        currNod = opened.pop(0)
        closed.append (currNod)
        # if I was the final state I should have already stopped
        for s in currNod.Succ():
            if s.info == scopInfo:
                done = True
                s.PrintPath()
                break

            if currNod.HasPred(s): continue
            if s.IsInList(closed) or s.IsInList(opened): continue
            alLeastOneBetter = False
            for i in range(n):
                if currNod.info[i] > s.info[i]:
                    alLeastOneBetter = True
            
            if not alLeastOneBetter: continue

            poz = len(opened)
            opened.append (s)

            while poz > 0 and opened[poz-1].f > opened[poz].f:
                opened[poz-1], opened[poz] = opened[poz], opened[poz-1]
                poz -= 1

            while poz > 0 and opened[poz-1].f == opened[poz].f and opened[poz-1].g > opened[poz].g:
                opened[poz-1], opened[poz] = opened[poz], opened[poz-1]
                poz -= 1

            if is_pressed('q'): # if it takes too long and you want to stop it
                invalidRsp = True
                while invalidRsp:
                    try:
                        rsp = int(input("Press 0 to exit or 1 to continue"))
                        if rsp == 1:
                            break
                        if rsp == 0:
                            done = True
                            print ("\tForce exit")
                            print ("Force exit", file=outputFile)
                            break
                        print(f'Invalid choice (choose between 0 and 1).')                            
                    except ValueError:
                            print(f'Invalid choice (choose between 0 and 1).')

    print (f"Duration: {time.time() - startTime}", file=outputFile)    
    if not done:
        print("Unsolvable", file=outputFile)

def main():
    inputFilesName = ["232_Surcea_Mihai-Daniel_Lab6_Pb8_input1.txt", "232_Surcea_Mihai-Daniel_Lab6_Pb8_input2.txt", "232_Surcea_Mihai-Daniel_Lab6_Pb8_input3.txt", "232_Surcea_Mihai-Daniel_Lab6_Pb8_input4.txt", "232_Surcea_Mihai-Daniel_Lab6_Pb8_input5.txt", "232_Surcea_Mihai-Daniel_Lab6_Pb8_input6.txt"]
    for inputFileName in inputFilesName:
        print ("\nUsing", inputFileName)
        readKeys(inputFileName)
        print ("\n\tUsing Hlocked")
        print ("\nUsing Hlocked", file=outputFile)
        solve (Hlocked)
        print ("\n\tUsing Hsum")
        print ("\nUsing Hsum", file=outputFile)
        solve (Hsum)
        print ("\n\tUsing Hmax")
        print ("\nUsing Hmax", file=outputFile)
        solve (Hmax)
        print ("\n\tUsing HH")
        print ("\nUsing HH", file=outputFile)
        solve (HH)
        outputFile.close()
    print("\nDone")

if __name__ == "__main__":
    main()



Using 232_Surcea_Mihai-Daniel_Lab6_Pb8_input1.txt

	Using Hlocked
	press q to stop the current execution

	Using Hsum
	press q to stop the current execution

	Using Hmax
	press q to stop the current execution

	Using HH
	press q to stop the current execution

Using 232_Surcea_Mihai-Daniel_Lab6_Pb8_input2.txt

	Using Hlocked
	press q to stop the current execution

	Using Hsum
	press q to stop the current execution

	Using Hmax
	press q to stop the current execution

	Using HH
	press q to stop the current execution

Using 232_Surcea_Mihai-Daniel_Lab6_Pb8_input3.txt

	Using Hlocked
	press q to stop the current execution

	Using Hsum
	press q to stop the current execution

	Using Hmax
	press q to stop the current execution

	Using HH
	press q to stop the current execution

Using 232_Surcea_Mihai-Daniel_Lab6_Pb8_input4.txt

	Using Hlocked
	press q to stop the current execution

	Using Hsum
	press q to stop the current execution

	Using Hmax
	press q to stop the current execution

	Using HH

In [19]:
def IsInList (self, nods):
        for nodPoz in range(len(nods)):
            
            # if nods[nodPoz].info == self.info: 
            #     return nodPoz

            atLeastOneBetter = False
            for i in range(n):
                if self.info[i] < nods[nodPoz].info[i]:
                    atLeastOneBetter = True        
                    break
            if not atLeastOneBetter: 
                return nodPoz #if nods[nodPoz].info == self.info else None
        return None